In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Кластеризация (Clustering)

Кластеризация — это задача объединения выборок в группы схожих выборок в соответствии с некоторой заранее определенной мерой сходства или несходства (например, на основе евклидова расстояния).
В этом разделе мы рассмотрим базовую задачу кластеризации на некоторых синтетических и реальных наборах данных.

Вот некоторые распространенные применения алгоритмов кластеризации:

- Сжатие в смысле сокращения данных.
- Использование в качестве этапа предварительной обработки для рекомендательных систем.
- Например:
    - группировка связанных веб-новостей (например, новостей Google) и результатов веб-поиска.
    - группировка связанных котировок акций для управления инвестиционным портфелем
    - составление профилей клиентов для анализа рынка.
- Создание кодовой книги образцов прототипов для неконтролируемого извлечения признаков.

Начнем с очень простого и очевидного примера:

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(random_state=42)
X.shape

In [ ]:
plt.scatter(X[:, 0], X[:, 1])
plt.show()

В данных явно присутствуют три отдельные группы точек, и мы хотели бы выделить их с помощью кластеризации.
Даже если группы в данных очевидны, их трудно найти, когда данные находятся в многомерном пространстве.

Воспользуемся одним из самых простых алгоритмов кластеризации — K-means.
Это итерационный алгоритм, который ищет три центра кластера так, чтобы расстояние от каждой точки до центра ее кластера было минимальным.

**Вопрос:** как будет выглядеть результат?

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)

Мы можем получить метки кластеров либо вызвав `fit`, а затем обратившись к атрибуту `labels_` объекта KMeans, либо вызвав `fit_predict`. В любом случае результат содержит идентификатор кластера, которому назначена каждая точка.

In [ ]:
labels = kmeans.fit_predict(X)

In [ ]:
labels

In [ ]:
all(labels == kmeans.labels_)

Посмотрим, что получилось

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=labels)
plt.show()

Здесь нас, вероятно, устраивает кластеризация. Но в целом нам, возможно, захочется получить количественную оценку. Можно сравнить метки наших кластеров с фактическими данными, которые мы получили при генерации больших двоичных объектов.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(y, labels))
print(confusion_matrix(y, labels))


In [ ]:
np.mean(y == labels)

Несмотря на то, что мы прекрасно восстановили разбиение данных на кластеры, назначенные нами идентификаторы кластеров были произвольными, и мы не можем надеяться на их повторяемость. Следовательно, необходимо использовать другую метрику оценки, например, ``adjusted_rand_score``, которая инвариантна к перестановкам меток:

In [ ]:
from sklearn.metrics import adjusted_rand_score
adjusted_rand_score(y, labels)

**Предположения кластеризации:** алгоритм кластеризации находит кластеры, делая предположения о том, что выборки должны быть сгруппированы вместе. Каждый алгоритм делает разные предположения, и качество и интерпретируемость ваших результатов будут зависеть от того, удовлетворяются ли эти предположения вашей цели. Для кластеризации K-средних модель заключается в том, что все кластеры имеют одинаковую сферическую дисперсию.

**В общем, нет никакой гарантии, что структура, найденная алгоритмом кластеризации, имеет какое-либо отношение к тому, что вас интересовало.**

Мы можем легко создать набор данных с неизотропными кластерами, на которых kmeans не сработает:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=170, n_samples=600)
rng = np.random.RandomState(74)

transformation = rng.normal(size=(2, 2))
X = np.dot(X, transformation)

y_pred = KMeans(n_clusters=3, n_init=10).fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred)
plt.show()

## Некоторые полезные процедуры кластеризации

Ниже приведены несколько хорошо известных алгоритмов кластеризации. 

- `sklearn.cluster.KMeans`: <br/>
    Самый простой, но эффективный алгоритм кластеризации. Необходимо заранее указать количество кластеров, и предполагается, что входные данные нормализованы (в качестве препроцессора используется модель PCA).
- `sklearn.cluster.MeanShift`: <br/>
    Может найти более привлекательные кластеры, чем KMeans, но не масштабируется для большого количества выборок.
- `sklearn.cluster.DBSCAN`: <br/>
    Может обнаруживать кластеры неправильной формы на основе компактности, т. е. разреженные области во входном пространстве, скорее всего, станут границами между кластерами. Также может обнаруживать выбросы (выборки, не являющиеся частью кластера).
- `sklearn.cluster.AffinityPropagation`: <br/>
    Алгоритм кластеризации, основанный на передаче сообщений между точками данных.
- `sklearn.cluster.SpectralClustering`: <br/>
    KMeans применяется к проекции лапласиана нормализованного графа: находит нормализованные разрезы графа, если матрица близости интерпретируется как матрица смежности графа.
- `sklearn.cluster.Ward`: <br/>
    Ward реализует иерархическую кластеризацию на основе алгоритма Уорда, подхода, минимизирующего дисперсию. На каждом шаге он минимизирует сумму квадратов разностей внутри всех кластеров (критерий инерции).

Из них Ward, SpectralClustering, DBSCAN и метод распространения близости (affinity propagation) также могут работать с предварительно вычисленными матрицами сходства.

<img src="figures/cluster_comparison.png" width="900">

Упражнение
============

Для вышеиспользуемого набора данных с неизотропными кластерами, на которых kmeans не работает, найдите алгоритм кластеризации, который хорошо выделяет очевидные кластеры.

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=170, n_samples=600)
rng = np.random.RandomState(74)

